In [108]:
% matplotlib inline
import seaborn as sns
import numpy as np
import pylab as pl
import mahotas as mh
import skimage as ski
import skimage.filters as skif
from scipy import ndimage as ndi
import matplotlib.pyplot as py
import pandas as pd
from skimage import segmentation
from skimage import morphology
from pprint import pprint
from sklearn import cluster
from sklearn import decomposition

In [119]:

def labelcells(image,kernelsize):
    '''
    Takes an image and a size for the kernel used to blur the distance matrix. Returns a trimed/blob-labeled image.
    '''
    img2=np.mean(image,2).astype(u'uint8')
    smooth = img2#mh.gaussian_filter(img2,1.5).astype('uint8')
    thresh = mh.thresholding.otsu(smooth)
    mask = smooth>thresh
    inverted = np.invert(smooth>thresh)
    filled = ndi.binary_fill_holes(inverted)
    dist = mh.distance(filled)
    #dist = ndi.morphology.distance_transform_edt(filled)
    dist = dist.max() - dist
    dist -= dist.min()
    dist  = dist/float(dist.ptp()) * 255
    dist = dist.astype(np.uint8)
    smooth2 = mh.gaussian_filter(dist,kernelsize).astype('uint8')
    rmax = mh.regmin(smooth2)
    seeds, ncells = mh.label(rmax)
    nuclei = mh.cwatershed(smooth2,seeds)
    whole = (nuclei*filled)
    return trimedges(whole)
    

def trimedges(whole):
    '''
    Takes a masked/labeled image and returns it with any object touching the edge to 
    '''
    borders = np.zeros(whole.shape,np.bool)
    borders[0:1,:] = 1
    borders[-1:,:] = 1
    borders[:,0:1] = 1
    borders[:,-1:] = 1
    at_border = np.unique(whole[borders])
    for obj in at_border:
        whole[whole==obj] = 0
    return whole

def separatecells(labeled,image):
    cellimages = []
    ylen,xlen = labeled.shape
    x = []
    y = []
    area = []
    for cell in np.unique(labeled)[1:]:
        cellmask = labeled.copy()
        cellmask[cellmask!=cell]=0
        cellmask[cellmask==cell]=1
        xmin = np.nonzero(cellmask.sum(axis = 0))[0].min()-5
        xmax = np.nonzero(cellmask.sum(axis = 0))[0].max()+5
        ymin = np.nonzero(cellmask.sum(axis = 1))[0].min()-5
        ymax = np.nonzero(cellmask.sum(axis = 1))[0].max()+5
        if xmin<0:
            xmin = 0
        if xmax>xlen:
            xmax = xlen
        if ymin<0:
            ymin = 0
        if ymax>ylen:
            ymax = ylen
        #takes care of cells close to edge that aren't touching it. Otherwise the pulled image won't work
        crop = image[ymin:ymax,xmin:xmax]
        cropmask = cellmask[ymin:ymax,xmin:xmax]
        cropmask = ndi.binary_dilation(cropmask,iterations=2)
        crop2 = np.empty(crop.shape)
        crop2[:,:,0] = cropmask*crop[:,:,0]
        crop2[:,:,1] = cropmask*crop[:,:,1]
        crop2[:,:,2] = cropmask*crop[:,:,2]
        crop2 = crop2.astype('uint8')
        cellimages.append((crop2))
        yl,xl, d =crop.shape
        x.append(xmax-xmin) #xl
        y.append(ymax-ymin) #yl
        area.append(yl*xl)
    return cellimages

def image2cells(image):
    labeled = labelcells(image,1.9)
    return separatecells(labeled,image)

def channel2vec(img):
    vec =[]
    xavgs = img.mean(0)
    yavgs = img.mean(1)
    y,x = img.shape
    yslice = np.linspace(0,y-1,4).astype('int')
    xslice = np.linspace(0,x-1,4).astype('int')
    slic = 0
    for index in yslice[1:]:
        vec.append(yavgs[slic:index].mean())
        slic = index
    slic = 0
    for index in xslice[1:]:
        vec.append(yavgs[slic:index].mean())
        slic = index
    return vec
    
def cell2vec(image):
    vec = list(image.shape[:2])
    for chan in range(0,3):
        channel = img[:,:,chan]
        vec = vec + channel2vec(channel)
    return vec


In [115]:
img = mh.imread('blood2.jpg')

In [116]:
cells =imagetocells(img)

In [217]:
def image2vec(image):
    cells = image2cells(image)
    cellvecs = []#[range(1,len(cells)+1)]
    #i = 1
    #cellids = []
    for cell in cells:
        cellvec = cell2vec(cell)
        #cellvec.insert(0,i)
        cellvecs.append(cellvec)
        #i += 1
        #cellids.append((i,cell))
    df = pd.DataFrame(cellvecs)
    return df, cells

In [218]:
test, cells = image2vec(img)